<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/LangChain_AI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing necessary libraries
!pip install langchain-openai --quiet  # Installs a specific version (0.0.3) of the langchain_openai package quietly
!pip install langchain langchain-experimental --quiet        # Installs a specific version (0.1.1) of the langchain package quietly
!pip install huggingface_hub openai google-search-results tiktoken wikipedia --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import os
os.environ['OPENAI_API_KEY'] = "sk-xxxxxxxxxxxxxxxxxxxxxuYECXfHihuS"

## Zero-shot ReAct Agent

In [6]:
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)

from langchain.agents import Tool
tool_list = [Tool(name = "Math Tool", func=tools[0].run,
        description="Tool to calculate, nothing else",
        return_direct=True),]

from langchain.agents import initialize_agent

agent = initialize_agent(tool_list, llm,
                         agent="zero-shot-react-description", verbose=True)
agent.run("What is 100 divided by 25?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I can use the Math Tool to calculate the division.
Action: Math Tool
Action Input: 100 divided by 25
Observation: Answer: 4.0


> Finished chain.


'Answer: 4.0'

## COnversational ReAct Agent

In [9]:
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
tools = load_tools(["wikipedia"],  llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,tools=tools,
                                        llm=llm, verbose=True,  max_iterations=3, memory=memory)
output_1=conversational_agent.run("Who is this guy 'Elon Musk'?")
output_2=conversational_agent.run("which person we were taking about?")
print(output_1)
print("*"*100)
print(output_2)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: Elon Musk
```
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the second wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023, according to the Bloomberg Billionaires Index, and $182.6  billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born moth

## Structured Chat Agent

In [11]:
!pip install langchainhub --quiet

In [12]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage


# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")
tools = load_tools(["wikipedia"],  llm=llm)
# Choose the LLM that will drive the agent
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")
# Construct the JSON agent
agent = create_structured_chat_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke(
    {
        "input": "what's my name? Do not use tools unless you have to",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...
Could not parse LLM output: Understood. Your name is Bob.Invalid or incomplete response{
  "action": "Final Answer",
  "action_input": "Your name is Bob."
}

> Finished chain.


{'input': "what's my name? Do not use tools unless you have to",
 'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

# DocStore Agent

In [13]:
from langchain.agents import initialize_agent, Tool
from langchain_openai import ChatOpenAI
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer
#binitializing an LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# initializing a docstore explorer
docstore = DocstoreExplorer(Wikipedia())

# initializing a tool
tools=[
    Tool(name="Search", func=docstore.search, description="useful for when you need to ask with search"),
    Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to ask with lookup")]

react_agent= initialize_agent(tools, llm, agent="react-docstore",verbose=True)
print(react_agent.run("What was Mahatma Gandhi's full name?")) # look on the keywords then go for search

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.react.base.DocstoreExplorer` was deprecated in langchain 0.1.0 and will be removed in 0.2.0
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I need to search Mahatma Gandhi and find his full name.
Action: Search[Mahatma Gandhi]
Observation: Mohandas Karamchand Gandhi (ISO: Mōhanadāsa Karamacaṁda Gāṁdhī; 2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple in London, and was called to the bar in June 1891, at the age of 22. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1893 to represent an Indian merchant in a lawsuit. He went on to liv

## Self Ask with Search Agent

In [14]:
import os
os.environ['TAVILY_API_KEY'] = "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"

In [16]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilyAnswer
import os

tools = [TavilyAnswer(max_results=2, name="Intermediate Answer")]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Construct the Self Ask With Search Agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)
agent_executor.invoke(
    {"input": "What is the hometown of the reigning men's U.S. Open champion?"}
)



> Entering new AgentExecutor chain...
Yes.
Follow up: Who is the reigning men's U.S. Open champion?The reigning men's U.S. Open champion is Daniil Medvedev. He won the tournament in 2021 by defeating Novak Djokovic in the final.Follow up: Where is Daniil Medvedev from?Daniil Medvedev is from Russia.So the final answer is: Russia

> Finished chain.


{'input': "What is the hometown of the reigning men's U.S. Open champion?",
 'output': 'Russia'}